In [34]:
import librosa
import os
import json
from tqdm import tqdm
from math import *
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from copy import *
import soundfile as sf
import sounddevice as sd
from scipy.io.wavfile import write
import wavio as wv
from sklearn.metrics import *
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import IPython.display as display



In [2]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

/Users/vikasthapar/opt/anaconda3/lib/python3.8/site-packages/transformers-4.6.1-py3.8.egg/transformers/models/wav2vec2/tokenization_wav2vec2.py:417: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
file='/Users/vikasthapar/Desktop/TCS_KWS/personal_data_copy/moderator/9.wav'
data,sr = librosa.core.load(file)
print(data.shape,sr)

signal, sample_rate = librosa.load(file, sr=16000)
print(signal.shape,sample_rate)

sf.write('file3.wav', signal, sample_rate, 'PCM_16')





(33075,) 22050
(24000,) 16000


In [ ]:
class AudioAugmentation:
    def read_audio_file(self, file_path):
        input_length = 24000
        data = librosa.load(file_path,sr=16000)[0]
        if len(data) > input_length:
            data = data[:input_length]
        else:
            data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
        return data

    def write_audio_file(self, file, data, sample_rate=16000):
        sf.write(file, data, sample_rate, 'PCM_16')
#         librosa.output.write_wav(file, data, sample_rate)

    def plot_time_series(self, data):
        fig = plt.figure(figsize=(14, 8))
        plt.title('Raw wave ')
        plt.ylabel('Amplitude')
        plt.plot(np.linspace(0, 1, len(data)), data)
        plt.show()

    def add_noise(self, data, factor=0.005):
        noise = np.random.randn(len(data))
        data_noise = data + factor * noise
        return data_noise

    def shift(self, data, factor=1600):
        return np.roll(data, factor)
    
    def pitch_shift(self, data, sr=16000, n_steps=4):
        return librosa.effects.pitch_shift(data, sr, n_steps=n_steps)

#     def stretch(self, data, rate=1):
#         input_length = 24000
#         data = librosa.effects.time_stretch(data, rate)
#         if len(data) > input_length:
#             data = data[:input_length]
#         else:
#             data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
#         return data

In [4]:
DATASET_PATH="/Users/vikasthapar/Desktop/TCS_KWS/personal_data_copy"
JSON_PATH='/Users/vikasthapar/Desktop/TCS_KWS/data2.json'
SAMPLES_TO_CONSIDER = 33075

In [ ]:
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(DATASET_PATH)):
    if dirpath is not DATASET_PATH:
        for f in tqdm(filenames):
            file_path = os.path.join(dirpath, f)
            aa = AudioAugmentation()
            data = aa.read_audio_file(file_path)
            
            dot=file_path.rfind('.wav')
            
#             aa.write_audio_file(file_path[:dot]+"_normal"+".wav", data)


#             data_noise = aa.add_noise(data, 0.006)
#             aa.write_audio_file(file_path[:dot]+"_noise1"+".wav", data_noise)
            
#             data_noise = aa.add_noise(data, 0.005)
#             aa.write_audio_file(file_path[:dot]+"_noise2"+".wav", data_noise)
            
            data_noise = aa.add_noise(data, 0.004)
            aa.write_audio_file(file_path[:dot]+"_noise3"+".wav", data_noise)

#             data_noise = aa.add_noise(data, 0.002)
#             aa.write_audio_file(file_path[:dot]+"_noise4"+".wav", data_noise)



            # data_roll = aa.shift(data)
            # aa.write_audio_file(file_path[:dot]+"_shift1"+".wav", data_roll)

#             data_roll = aa.shift(data,400)
#             aa.write_audio_file(file_path[:dot]+"_shift1"+".wav", data_roll)

#             data_roll = aa.shift(data,200)
#             aa.write_audio_file(file_path[:dot]+"_shift2"+".wav", data_roll)


            data_pitch= aa.pitch_shift(data, sr=16000, n_steps=-1)
            aa.write_audio_file(file_path[:dot]+"_pitch1"+".wav", data_pitch)

#             data_pitch= aa.pitch_shift(data, sr=16000, n_steps=2)
#             aa.write_audio_file(file_path[:dot]+"_pitch2"+".wav", data_pitch)
                
            data_pitch= aa.pitch_shift(data, sr=16000, n_steps=3)
            aa.write_audio_file(file_path[:dot]+"_pitch3"+".wav", data_pitch)
                
#             data_pitch= aa.pitch_shift(data, sr=16000, n_steps=4)
#             aa.write_audio_file(file_path[:dot]+"_pitch4"+".wav", data_pitch)
                
                

#             data_stretch = aa.stretch(data, 1.1)
#             aa.write_audio_file(file_path[:dot]+"_stretch1"+".wav", data_stretch)

#             data_stretch = aa.stretch(data, 0.8)
#             aa.write_audio_file(file_path[:dot]+"_stretch2"+".wav", data_stretch)

In [6]:
data = {
        "mapping": [],
        "labels": [],
        "token_input": [],
        "files": []
    }

for i, (dirpath, dirnames, filenames) in enumerate(os.walk(DATASET_PATH)):

    if dirpath is not DATASET_PATH:

        label = dirpath.split("/")[-1]
        data["mapping"].append(label)
        print("\nProcessing: '{}'".format(label))

        for f in tqdm(filenames):
            file_path = os.path.join(dirpath, f)

            signal, sample_rate = librosa.load(file_path,sr=16000)
            

#             if len(signal) >= SAMPLES_TO_CONSIDER:

#                 signal = signal[:SAMPLES_TO_CONSIDER]
            input_values = tokenizer(signal, return_tensors = 'pt').input_values
            data["token_input"].append(input_values.T.tolist())
            data["labels"].append(i-1)
            data["files"].append(file_path)
#             print(signal.shape, input_values.shape,file_path)
#             break
            
#         break
            
    
    
    
    
    
            
            
                
#                 MFCCs = librosa.feature.mfcc(signal, sample_rate, n_mfcc=13, n_fft=2048,
#                                                 hop_length=512)
                
#                 data["MFCCs"].append(MFCCs.T.tolist())
#                 data["labels"].append(i-1)
#                 data["files"].append(file_path)

  0%|          | 0/10 [00:00<?, ?it/s]


Processing: 'moderator'


 30%|███       | 3/10 [00:00<00:00, 16.07it/s]


Processing: 'activation'


 40%|████      | 4/10 [00:00<00:00, 33.12it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]


Processing: 'inspiration'

Processing: 'monument'


  5%|▌         | 4/80 [00:00<00:02, 35.73it/s]


Processing: 'motivation'


100%|██████████| 80/80 [00:03<00:00, 25.11it/s]


In [ ]:
display.Audio('/Users/vikasthapar/Desktop/TCS_KWS/personal_data_copy/moderator/9.wav', autoplay=True)



In [7]:
len(data['token_input'])

160

In [8]:
with open(JSON_PATH, "w") as fp:
    json.dump(data, fp, indent=4)


In [75]:
with open(JSON_PATH, "r") as fp:
        data = json.load(fp)

X = np.array(data["token_input"])
y = np.array(data["labels"])
X.shape,y.shape

((160, 24000, 1), (160,))

In [76]:
MAPPINGS=data["mapping"]
MAPPINGS

['moderator',
 'activation',
 'motorbike',
 'magician',
 'dedication',
 'monsoon',
 'inspiration',
 'monument',
 'motivation']

In [77]:
X=X.reshape(160,24000//24,24,1)
X.shape,y.shape

((160, 1000, 24, 1), (160,))

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, shuffle=True)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train,shuffle=True)

# X_train = X_train[..., np.newaxis]
# X_test = X_test[..., np.newaxis]
# X_validation = X_validation[..., np.newaxis]

In [79]:
X_train=np.repeat(X_train,10, axis=0)
y_train=np.repeat(y_train,10)

In [80]:
X_train.shape,y_train.shape

((1150, 1000, 24, 1), (1150,))

In [81]:
X_train.shape[1], X_train.shape[2], 1

(1000, 24, 1)

In [82]:
input_shape = (X_train.shape[1], X_train.shape[2], 1)

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))


model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))

model.add(tf.keras.layers.Conv2D(32, (2, 2), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
tf.keras.layers.Dropout(0.3)

model.add(tf.keras.layers.Dense(9, activation='softmax'))

model.summary()



Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 998, 22, 64)       640       
_________________________________________________________________
batch_normalization_9 (Batch (None, 998, 22, 64)       256       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 499, 11, 64)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 497, 9, 32)        18464     
_________________________________________________________________
batch_normalization_10 (Batc (None, 497, 9, 32)        128       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 249, 5, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 248, 4, 32)       

In [83]:
optimiser = tf.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimiser,
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"])

In [84]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", min_delta=0.001, patience=5)

history = model.fit(X_train,
                    y_train,
                    epochs=10,
                    batch_size=8,
                    validation_data=(X_validation, y_validation),
                    callbacks=[earlystop_callback])

Epoch 1/10
144/144 [==============================] - 56s 383ms/step - loss: 0.9201 - accuracy: 0.7799 - val_loss: 1.6146 - val_accuracy: 0.6207
Epoch 2/10
144/144 [==============================] - 55s 379ms/step - loss: 0.0833 - accuracy: 1.0000 - val_loss: 1.2300 - val_accuracy: 0.6552
Epoch 3/10
144/144 [==============================] - 55s 383ms/step - loss: 0.0793 - accuracy: 1.0000 - val_loss: 1.1752 - val_accuracy: 0.7931
Epoch 4/10
144/144 [==============================] - 56s 386ms/step - loss: 0.0779 - accuracy: 1.0000 - val_loss: 1.1978 - val_accuracy: 0.8276
Epoch 5/10
144/144 [==============================] - 55s 383ms/step - loss: 0.0769 - accuracy: 1.0000 - val_loss: 1.2118 - val_accuracy: 0.8276
Epoch 6/10
144/144 [==============================] - 57s 398ms/step - loss: 0.0762 - accuracy: 1.0000 - val_loss: 1.2203 - val_accuracy: 0.8276
Epoch 7/10
144/144 [==============================] - 57s 398ms/step - loss: 0.0755 - accuracy: 1.0000 - val_loss: 1.2392 - val_ac

In [85]:
model.save("/Users/vikasthapar/Desktop/TCS_KWS/trying_mix.h5")

In [23]:
freq = 16000
duration = 1.5
print("Speak now!")

recording = sd.rec(int(duration * freq), samplerate=freq, channels=1, dtype='int16')
sd.wait()
write("file2.wav", freq, recording)

Speak now!


In [24]:
speech, rate = librosa.load("file2.wav",sr=16000)

In [25]:
import IPython.display as display
display.Audio("file2.wav", autoplay=True)

In [47]:
input_values = tokenizer(speech, return_tensors = 'pt').input_values

In [48]:
input_values.shape

torch.Size([1, 24000])

In [49]:
input_values=np.array(input_values.T.tolist())
# input_values.shape
len(input_values)


24000

In [50]:
input_values=input_values.reshape(1,24000//24,24,1)
input_values.shape

(1, 1000, 24, 1)

In [51]:
prob = model.predict(input_values)
prob
# predicted_ids = torch.argmax(logits, dim =-1)



array([[0.00933649, 0.09079586, 0.01537031, 0.0667679 , 0.2420502 ,
        0.16342832, 0.02312141, 0.03501501, 0.35411453]], dtype=float32)

In [53]:
predicted_index = np.argmax(prob)
predicted_keyword = MAPPINGS[predicted_index]
predicted_keyword

'motivation'